# Data Science

### GBR & XGBR

When I merge the hpg relavant dataset and air relavant dataset, there is a lot of missing data since the store id is not match completely. However, the NN could figure out the missing data which is data mining in the data which include the missing data directly. Normally, the system set the null number as -1 to present this value is not exist. So, in the data preprocessing part, i refill the missing data to -1. Then the NN could handle it.

In [1]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor
from keras.layers import Embedding, Input, Dense
from keras.models import Model
import keras
import keras.backend as K
import matplotlib.pyplot as plt

D:\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\anaconda\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
D:\anaconda\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)
Using TensorFlow backend.


In [2]:
#import data
train=pd.read_csv('E:/Foundations of Data Science/Referral/Forecast customer/Data/PreparedData/train.csv')
test=pd.read_csv('E:/Foundations of Data Science/Referral/Forecast customer/Data/PreparedData/test.csv')

In [3]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

def plot_actual_predicted(actual, predicted):
    print('RMSE: ', RMSLE(actual, predicted))
    tmp = pd.DataFrame({'actual': actual, 'predicted': predicted}).sort_values(['actual'])
    plt.scatter(range(tmp.shape[0]), tmp['predicted'], color='green')
    plt.scatter(range(tmp.shape[0]), tmp['actual'], color='blue')
    plt.show()
    del tmp

In [4]:
# Set the Train and test data prepared for NN
value_col = ['holiday_flg','min_visitors','mean_visitors','median_visitors','max_visitors','count_observations',
'rs1_x','rv1_x','rs2_x','rv2_x','rs1_y','rv1_y','rs2_y','rv2_y','total_reserv_sum','total_reserv_mean',
'total_reserv_dt_diff_mean','date_int','var_max_lat','var_max_long','lon_plus_lat']

nn_col = value_col + ['dow', 'year', 'month', 'air_store_id2', 'air_area_name', 'air_genre_name',
'air_area_name0', 'air_area_name1', 'air_area_name2', 'air_area_name3', 'air_area_name4',
'air_area_name5', 'air_area_name6', 'air_genre_name0', 'air_genre_name1',
'air_genre_name2', 'air_genre_name3', 'air_genre_name4']

X = train.copy()
X_test = test[nn_col].copy()

value_scaler = preprocessing.MinMaxScaler()
for vcol in value_col:
    X[vcol] = value_scaler.fit_transform(X[vcol].values.astype(np.float64).reshape(-1, 1))
    X_test[vcol] = value_scaler.transform(X_test[vcol].values.astype(np.float64).reshape(-1, 1))

X_train = list(X[nn_col].T.as_matrix())
Y_train = X['visitors'].values
nn_train = [X_train, Y_train]
nn_test = [list(X_test[nn_col].T.as_matrix())]


In [ ]:
col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
model1 = ensemble.GradientBoostingRegressor(learning_rate=0.2, random_state=3,
                    n_estimators=180, subsample=0.78, max_depth=10)
# model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)
# model2 = ensemble.RandomForestRegressor(n_estimators=13, random_state=3, max_depth=18,
#                                         min_weight_fraction_leaf=0.0002)
model3 = XGBRegressor(learning_rate=0.2, random_state=3, n_estimators=330, subsample=0.8, 
                      colsample_bytree=0.8, max_depth=10)

model1.fit(train[col], train['visitors'].values)
print("Model1 trained")

model3.fit(train[col], train['visitors'].values)
print("Model3 trained")


preds1 = model1.predict(train[col])
preds3 = model3.predict(train[col])

actual_output = train['visitors'].values
print('GradientBoostingRegressor:')
plot_actual_predicted(actual_output, preds1)
print('XGBRegressor:')
plot_actual_predicted(actual_output, preds3)

preds1 = model1.predict(test[col])
preds3 = model3.predict(test[col])

test['visitors'] =preds3
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
sub1 = test[['id','visitors']].copy()
print("Model predictions done.")

Model1 trained
